In [1]:
import duckdb
pytable  = duckdb.sql("SELECT * FROM read_parquet('https://www.data.gouv.fr/fr/datasets/r/ff16d511-10c0-405e-9b35-511723948fce')").arrow()

In [2]:
pytable

pyarrow.Table
id_election: string
id_brut_miom: string
Code du département: string
Libellé du département: string
Code de la commune: string
Libellé de la commune: string
Code du b.vote: string
Inscrits: int32
Abstentions: int32
% Abs/Ins: float
Votants: int32
% Vot/Ins: float
Blancs: int32
% Blancs/Ins: float
% Blancs/Vot: float
Nuls: int32
% Nuls/Ins: float
% Nuls/Vot: float
Exprimés: int32
% Exp/Ins: float
% Exp/Vot: float
Code de la circonscription: string
Libellé de la circonscription: string
Code du canton: string
Libellé du canton: string
----
id_election: [["2019_euro_t1","2019_euro_t1","2019_euro_t1","2019_euro_t1","2019_euro_t1",...,"2015_regi_t1","2015_regi_t1","2015_regi_t1","2015_regi_t1","2015_regi_t1"],["2015_regi_t1","2015_regi_t1","2015_regi_t1","2015_regi_t1","2015_regi_t1",...,"2007_pres_t2","2007_pres_t2","2007_pres_t2","2007_pres_t2","2007_pres_t2"],["2007_pres_t2","2007_pres_t2","2007_pres_t2","2007_pres_t2","2007_pres_t2",...,"2014_muni_t2","2014_muni_t2","2014_m

In [3]:
duckdb.sql("SELECT count(*) FROM pytable")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│      2843568 │
└──────────────┘

In [4]:
agg = duckdb.sql("SELECT sum(Abstentions) AS sum_abs, id_election FROM pytable GROUP BY id_election ORDER BY sum_abs")
print(agg)

┌──────────┬──────────────┐
│ sum_abs  │ id_election  │
│  int128  │   varchar    │
├──────────┼──────────────┤
│  5571415 │ 2004_cant_t2 │
│  5798313 │ 2008_cant_t2 │
│  6234939 │ 2001_cant_t2 │
│  6561462 │ 2007_pres_t2 │
│  6615286 │ 2007_pres_t1 │
│  6761060 │ 2001_cant_t1 │
│  6943318 │ 2014_muni_t2 │
│  7221945 │ 2004_cant_t1 │
│  7520183 │ 2008_cant_t1 │
│  7979673 │ 2002_pres_t2 │
│     ·    │      ·       │
│     ·    │      ·       │
│     ·    │      ·       │
│ 25697206 │ 2020_muni_t1 │
│ 25697541 │ 2022_legi_t1 │
│ 26125307 │ 2022_legi_t2 │
│ 26283594 │ 2009_euro_t1 │
│ 26796819 │ 2014_euro_t1 │
│ 27128488 │ 2017_legi_t2 │
│ 27452627 │ 2021_dpmt_t2 │
│ 28838758 │ 2021_dpmt_t1 │
│ 30000907 │ 2021_regi_t2 │
│ 30644259 │ 2021_regi_t1 │
├──────────┴──────────────┤
│   49 rows (20 shown)    │
└─────────────────────────┘



In [9]:
from pyspark.sql import SparkSession
iceberg_builder = SparkSession.builder \
    .appName("iceberg-spark-minio-example") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4,org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.5.0,org.apache.iceberg:iceberg-hive-runtime:1.5.0") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.spark_catalog.type", "hadoop") \
    .config("spark.sql.catalog.spark_catalog.warehouse", "s3a://iceberg/iceberg_data/") \
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio-api.punch:8080") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .enableHiveSupport()


In [10]:
spark = configure_spark_with_delta_pip(builder, extra_packages=["org.apache.hadoop:hadoop-aws:3.3.4"]).getOrCreate()

24/06/13 23:54:43 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [13]:
# Read CSV file
csv_path = '/home/dorian/punchplatform/duckdb/deltalake/election.csv'  # Update this path to the actual location of your CSV file
df = spark.read.format('csv').option('header', 'true').option('inferSchema', 'true').load(csv_path)
file_count = df.count()


Py4JJavaError: An error occurred while calling o64.load.
: org.apache.spark.SparkException: Cannot find catalog plugin class for catalog 'spark_catalog': org.apache.iceberg.spark.SparkCatalog.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.catalogPluginClassNotFoundForCatalogError(QueryExecutionErrors.scala:1925)
	at org.apache.spark.sql.connector.catalog.Catalogs$.load(Catalogs.scala:70)
	at org.apache.spark.sql.connector.catalog.CatalogManager.loadV2SessionCatalog(CatalogManager.scala:67)
	at org.apache.spark.sql.connector.catalog.CatalogManager.$anonfun$v2SessionCatalog$2(CatalogManager.scala:86)
	at scala.collection.mutable.HashMap.getOrElseUpdate(HashMap.scala:86)
	at org.apache.spark.sql.connector.catalog.CatalogManager.$anonfun$v2SessionCatalog$1(CatalogManager.scala:86)
	at scala.Option.map(Option.scala:230)
	at org.apache.spark.sql.connector.catalog.CatalogManager.v2SessionCatalog(CatalogManager.scala:85)
	at org.apache.spark.sql.connector.catalog.CatalogManager.catalog